In [60]:
import globus_sdk
import requests
import json

# Globus ID and Scope
# ===================

# Define your Globus thick-client ID
# https://app.globus.org/settings/developers
# This needs to be created by users (they can also re-use one of the existing clients)
my_client_id = "99bc7f15-ebed-4d6b-b783-8f7ae8735135"

# Define the inference-gateway resource service scope
# This will be publicly available to users
vllm_client_id = "681c10cc-f684-4540-bcd7-0b4df3bc26ef"
vllm_scope = f"https://auth.globus.org/scopes/{vllm_client_id}/action_all"

# Authentication and Access Token
# ===============================

# Start an Auth client with the vLLM scope
auth_client = globus_sdk.NativeAppAuthClient(my_client_id)
auth_client.oauth2_start_flow(requested_scopes=vllm_scope)

# Authenticate with your Globus account
authorize_url = auth_client.oauth2_get_authorize_url()
print(f"Please go to this URL and login:\n\n{authorize_url}\n")


Please go to this URL and login:

https://auth.globus.org/v2/oauth2/authorize?client_id=99bc7f15-ebed-4d6b-b783-8f7ae8735135&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=https%3A%2F%2Fauth.globus.org%2Fscopes%2F681c10cc-f684-4540-bcd7-0b4df3bc26ef%2Faction_all&state=_default&response_type=code&code_challenge=cQ3ppKk7AwgHu22g5OrfQyt3GlFF90nEHUUYPquNLOo&code_challenge_method=S256&access_type=online



In [61]:
# Collect access token to vLLM service
auth_code = "YWuJ0iVyV0oPKnegIXxAMeYN7Vwk0n"
token_response = auth_client.oauth2_exchange_code_for_tokens(auth_code)
access_token = token_response.by_resource_server[vllm_client_id]["access_token"]

In [65]:
# URL to the inference gateway (needs to end with forward slash /)
#url = "https://data-portal-dev-vmw-01.cels.anl.gov/resource_server/polaris/"
url = "http://localhost:8000/resource_server/polaris/llama-cpp/completions/"
# Add access token to the headers
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

# Prepare the inference request
data = {
        "model": "meta-llama3-70b-instruct",
        "temperature": 0.2,
        "max_tokens": 150,
        "prompt": "List all proteins that interact with RAD51"    
}

# Convert data into Json51633660

data_json = json.dumps(data)

# Send the post request to the relay server
# Verify=False is temporary since I use a self-signed certificate
response = requests.post(url, data=data_json, headers=headers, verify=False)

# Print inference response
print(response.status_code)
print(response.json())

KeyboardInterrupt: 